In [ ]:
#this script gets all reports from 2016-2017 field season (s16)

#problem: reports are grouped by categories (eva, commander, etc) and not by groups. I can not automatically get the crew #. but I can get dates and later assign crew names using them later.  

In [ ]:
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
import urllib.request
import csv
import pandas as pd

In [ ]:

#different report landing pages; highest page # is 10.
# #first page does not have page number; ../page/1 gives 404 error.  
#start from 2; then add first pages to it (categories + good_links)

categories = ['http://mdrs2016.marssociety.org/category/daily-summary', 
'http://mdrs2016.marssociety.org/category/eva',
'http://mdrs2016.marssociety.org/category/commander',
'http://mdrs2016.marssociety.org/category/journalist',
'http://mdrs2016.marssociety.org/category/science',
'http://mdrs2016.marssociety.org/category/greenhab-report',
'http://mdrs2016.marssociety.org/category/creative-report'
]








# url = urllib.request.urlopen('http://mdrs2016.marssociety.org/category/eva').read()
# soup = bs(url, 'lxml')

In [ ]:
#loop thru categories, try to open pages and get all working links to reports (../page/#)

good_links = []
brokenlinks = [] #for checking whether good links were not thrown out
for c in categories:
    for i in range (2,11): # last page 11 
        try:
            url = urllib.request.urlopen(c + "/page/" + str(i))
            good_links.append(c + "/page/" + str(i))
        except urllib.error.HTTPError as exception: #404 error exception 
            brokenlinks.append(c + "/page/" + str(i))
        print (i)
        


In [ ]:

good_l = good_links  + categories
good_l

In [ ]:
# getting date; other parts of soup are the same as 2017-2019 data.  


# #soup.article.select(".entry-content
# #class_list = ["fa fa-clock-o"]; need to get text after for the date. 


# for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
#     comment = soup.article.find_all(text=lambda text: isinstance(text, Comment))[1]
     

# # comment = soup.article.find_all(text=lambda text: isinstance(text, Comment))[1]


# # comment = soup.find(text=lambda text:isinstance(text, Comment))
# commentsoup = bs(comment , 'lxml')

# #commentsoup.p.span.find_next
# #commentsoup.p.find(class_="fa fa-clock-o").next_sibling



In [ ]:
def parse(soup):
    date=[]
    header = []
    body = [] 
    html = []
    for i in range(len(soup.find_all("article"))):
        comment = soup.find_all("article")[i].find_all(text=lambda text: isinstance(text, Comment))[1]
        commentsoup = bs(comment, 'lxml')
        date.append (commentsoup.p.find(class_="fa fa-clock-o").next_sibling)
        header.append(soup.find_all("article")[i].select(".entry-title")[0].get_text())
        body.append(soup.find_all("article")[i].select(".entry-content")[0].get_text())
        html.append(soup.find_all("article")[i].select(".entry-content")[0])
    return date, header, body, html 




In [ ]:
date=[]
header =[]
body=[]
html =[]
link = []


for l in good_l:
    print(l)
    url = urllib.request.urlopen(l).read()
    soup = bs(url, 'lxml')
    for i in range(len(soup.find_all("article"))):
        comment = soup.find_all("article")[i].find_all(text=lambda text: isinstance(text, Comment))[1]
        commentsoup = bs(comment, 'lxml')
        date.append (commentsoup.p.find(class_="fa fa-clock-o").next_sibling)
        header.append(soup.find_all("article")[i].select(".entry-title")[0].get_text())
        body.append(soup.find_all("article")[i].select(".entry-content")[0].get_text())
        html.append(soup.find_all("article")[i].select(".entry-content")[0])
        link.append(l)


                    
    

In [ ]:
#check if all correct:
len(date), len(body), len(link), len(html), len(header)


date[0:10], body[0:10], link[0:10]

In [ ]:
#create panda dataframe and dump to csv

reports = pd.DataFrame({'date': date, 'header': header, 'body':body, 'html': html, 'link': link})

#sort by date 
reports['date'] =pd.to_datetime(reports.date)

sorted = reports.sort_values(by = ['date'])

sorted.to_csv(r's16_reports.csv')
